# Session 03: Intro to LLM Agents

In [1]:
# Optional: install dependencies
#!pip install openai-agents

In [2]:
import json
from IPython.display import display, Markdown

def mdprint(text):
    """Helper function for printing markdown text."""
    display(Markdown(text))

def pprint(result):
    """Helper function for pretty-printing raw model responses."""
    for item in result.new_items:
        print(item.__class__, json.dumps(item.to_input_item(), indent=2))

In [3]:
from api_key import API_KEY

API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
#API_KEY = "<KEY>"
API_MODEL = "1 - GPT-OSS-120b - an open model released by OpenAI in August 2025" # Best for fast dev runs

In [4]:
from agents import AsyncOpenAI, set_tracing_disabled, OpenAIChatCompletionsModel, Agent, Runner, ModelSettings
from openai.types.shared import Reasoning
# Disable the tracing feature
set_tracing_disabled(True)

# Instantiate the model with custom endpoint
model = OpenAIChatCompletionsModel(
    model=API_MODEL,
    openai_client=AsyncOpenAI(api_key=API_KEY, base_url=API_URL)
)

### Base Agent

The most basic agent is just an LLM call, without any further specifications.

In [5]:
input = "Whats the weather like in Kassel?"

In [6]:
base_agent = Agent(
    name="base_agent",
    model=model,
    model_settings=ModelSettings(
        reasoning=Reasoning(effort="low")
    )
)
base_result = await Runner.run(base_agent, input=input)

In [7]:
mdprint(base_result.final_output)
#pprint(result)

I’m sorry, but I don’t have the ability to look up real‑time weather information. You can check the current conditions in Kassel on a weather website or app such as Weather.com, AccuWeather, or a mobile weather app.

### Reasoning Agent

The base agent can be improved upon by enabling it to reason; it will usually provide better responses by allowing it to "think before answering".

In [8]:
reasoning_agent = Agent(
    name="base_agent",
    model=model,
    model_settings=ModelSettings(
        reasoning=Reasoning(effort="high")
    )
)
reasoning_result = await Runner.run(reasoning_agent, input=input)

In [9]:
mdprint(reasoning_result.final_output)
#pprint(result)

I’m not able to pull live weather data, so I can’t give you the current conditions in Kassel right now. For up‑to‑date information you can check a weather service such as:

- **Weather.com** – just type “Kassel, Germany” into the search box.  
- **MeteoGroup / Weather Underground** – similar search works there.  
- **Your phone’s weather app** – most smartphones include a global search function.  
- **Google** – typing “Kassel weather” into the search bar will show the latest forecast at the top of the results page.

If you’re curious about what the weather is typically like in Kassel throughout the year, here’s a quick overview:

| Season | Typical Temperatures (°C) | Typical Conditions |
|--------|---------------------------|--------------------|
| **Winter** (Dec‑Feb) | 0 – 4 (often below freezing at night) | Cold, occasional snow, cloudy days |
| **Spring** (Mar‑May) | 5 – 15 | Variable – cool early spring, milder later, occasional rain showers |
| **Summer** (Jun‑Aug) | 15 – 25 (can reach 28 – 30 on hot days) | Generally mild to warm, mixed sunshine and rain |
| **Autumn** (Sep‑Nov) | 8 – 18 (cooling quickly in November) | Cool, increasing rain, colorful foliage |

For a precise, up‑to‑date forecast (hourly, 7‑day, precipitation chances, etc.), please use one of the online services listed above. Stay prepared and enjoy your time in Kassel!

### Tool Use

As we have seen before, LLMs could greatly benefit from being able to interact with the world to, for example, retrieve up-to-date data. This is achieved through *tools*.

Lets implement a basic weather information tool, based on the `wttr.in` API.

In [10]:
!curl 'wttr.in/Kassel?format=j1'

{
    "current_condition": [
        {
            "FeelsLikeC": "13",
            "FeelsLikeF": "55",
            "cloudcover": "0",
            "humidity": "67",
            "localObsDateTime": "2025-11-13 11:58 AM",
            "observation_time": "10:58 AM",
            "precipInches": "0.0",
            "precipMM": "0.0",
            "pressure": "1011",
            "pressureInches": "30",
            "temp_C": "14",
            "temp_F": "58",
            "uvIndex": "1",
            "visibility": "10",
            "visibilityMiles": "6",
            "weatherCode": "113",
            "weatherDesc": [
                {
                    "value": "Sunny"
                }
            ],
            "weatherIconUrl": [
                {
                    "value": ""
                }
            ],
            "winddir16Point": "SSW",
            "winddirDegree": "191",
            "windspeedKmph": "18",
            "windspeedMiles": "11"
        }
    ],
    "nearest_area": [
   

In [11]:
import requests
from typing import Any
from agents import Agent, Runner, function_tool

@function_tool
def get_weather(city: str) -> dict[str, Any]:
    """Retrieves the weather forecast for a specified location."""
    return requests.get(f"http://wttr.in/{city}?format=j1").json()


In [12]:
tool_agent = Agent(
    name="tool_agent",
    model=model,
    instructions="Always use the provided tools to solve the task given by the user. Provide very succint answers.",
    tools=[get_weather],
)
tool_result = await Runner.run(tool_agent, input=input)

In [13]:
mdprint(tool_result.final_output)

**Current weather in Kassel (Nov 13, 2025)**  
- **Condition:** Sunny, clear skies  
- **Temperature:** 14 °C (58 °F) – feels like 13 °C (55 °F)  
- **Humidity:** 67 %  
- **Wind:** 18 km/h (11 mph) from the SSW  
- **Pressure:** 1011 hPa  

**Today’s outlook:**  
- High 15 °C (59 °F), low 6 °C (43 °F)  
- Mostly sunny, light winds, no rain.

In [14]:
result = await Runner.run(tool_agent, input="Do i need a jacket when going outside in Kassel?")

In [15]:
mdprint(result.final_output)

Current temperature ≈ 14 °C (57 °F) with sunny skies and a light 18 km/h wind. It feels like 13 °C. A light jacket or sweater is recommended.

### Structured Outputs

For many workflows, it is helpful to have agents return their response in a structured format (most commonly JSON), to be able to parse it into a python data structure and interface with program flow. The `agents` package uses `pydantic` for data modeling internally, so we will opt for that as well.

**Note**: due to API limitations, we cannot use the `output_type` parameter of the `Agent` class directly, but have to emulate its behaviour through explicit prompting.

Let's implement a basic `Feedback` data model, consisting of a written feedback and a score enum, that we can use to control program flow later:

In [16]:
# This is for casting output types to a JSON schema we can supply to the model.
from pydantic.dataclasses import dataclass
from typing import Literal

@dataclass
class Feedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]

Pydantic also provides a handy way to generate an explicit JSON schema that models should conform to:

In [17]:
from pydantic import TypeAdapter

TypeAdapter(Feedback).json_schema()

{'properties': {'feedback': {'title': 'Feedback', 'type': 'string'},
  'score': {'enum': ['pass', 'needs_improvement', 'fail'],
   'title': 'Score',
   'type': 'string'}},
 'required': ['feedback', 'score'],
 'title': 'Feedback',
 'type': 'object'}

### LLM-as-a-judge (Adaptive Loops)

We can extend our agent workflow to include multiple agents in multiple roles. For example, consider a story writing task with two agents, with the following flow:
- The first agent generates an outline for a story
- The second agent judges the outline and provides feedback
- We loop until the judge is satisfied with the outline

Here, the structured output defined previously is needed: we can use the `score` property of the judges' output to either continue refining, or exit.

**Note**: pay attention to cap the number of iterations, either by prompting or with a hard limit. Its easy to get stuck in an infinite feedback loop otherwise!

In [18]:
story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input. "
        "Do not write a full story, just the outline. "
        "If there is any feedback provided, use it to improve the outline."
    ),
    model=model
)

evaluator = Agent(
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough. "
        "If it's not good enough, you provide feedback on what needs to be improved. "
        "Never give it a pass on the first try. "
        "After 5 attempts, you can give it a pass if the story outline is good enough - do not go for perfection. "
        "Reply in the given structured format, conforming exactly to its specification: "
        f"{TypeAdapter(Feedback).json_schema()}"
    ),
    model=model
)

In [20]:
input_items = [{"content": "A story about a rainy afternoon in Kassel.", "role": "user"}]
outlines = []
feedbacks = []

while True:
    story_outline_result = await Runner.run(story_outline_generator, input_items)
    input_items = story_outline_result.to_input_list()
    outlines.append(story_outline_result.final_output)

    evaluator_result = await Runner.run(evaluator, input_items)
    result = Feedback(**json.loads(evaluator_result.final_output)) # Cast raw response to feedback dataclass
    feedbacks.append(result.feedback)

    print(f"Evaluator score: {result.score}")

    if result.score == "pass":
        print("Story outline is good enough, exiting.")
        break

    print("Re-running with feedback")

    input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

Evaluator score: needs_improvement
Re-running with feedback
Evaluator score: pass
Story outline is good enough, exiting.


In [21]:
mdprint(feedbacks[0])

The outline nicely evokes the rainy mood of Kassel and gives a clear protagonist, setting, and a series of plot beats. However, the conflict feels generic and the stakes could be stronger—why does the hidden letter matter to Lena beyond a nice discovery? The motivations of the old vendor are vague, and the climax hinges on a convenient key without much tension. Adding deeper character arcs, clearer consequences if the key isn’t found, and more specific sensory details of the rain and Kassel landmarks would make the story more compelling. Consider tightening the inciting incident and giving the romance revelation a stronger thematic tie to Lena’s work as a conservator.

### Agents-as-tools

We have already considered basic programmatic tools. However, we extend on that and call other agents for tools, to delegate tasks from a main coordinating agent. This is as simple as calling the `.as_tool` function of an agent, and passing it to the main agent.

In [22]:
german_agent = Agent(
    name="german_agent",
    instructions="You translate the user's message to German",
    handoff_description="An english to german translator",
    model=model
)

french_agent = Agent(
    name="french_agent",
    instructions="You translate the user's message to French",
    handoff_description="An english to french translator",
    model=model
)

italian_agent = Agent(
    name="italian_agent",
    instructions="You translate the user's message to Italian",
    handoff_description="An english to italian translator",
    model=model
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools in order."
        "You never translate on your own, you always use the provided tools."
    ),
    tools=[
        german_agent.as_tool(
            tool_name="translate_to_german",
            tool_description="Translate the user's message to German",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
        italian_agent.as_tool(
            tool_name="translate_to_italian",
            tool_description="Translate the user's message to Italian",
        ),
    ],
    model=model
)

synthesizer_agent = Agent(
    name="synthesizer_agent",
    instructions="You inspect translations, correct them if needed, and produce a final concatenated response.",
    model=model
)

In [23]:
msg = "Translate 'It's raining cats and dogs in Kassel.' to german and french."

orchestrator_result = await Runner.run(orchestrator_agent, msg)
synthesizer_result = await Runner.run(synthesizer_agent, orchestrator_result.to_input_list())

mdprint(synthesizer_result.final_output)


**German:** In Kassel regnet es in Strömen.  

**French:** Il pleut des cordes à Kassel.

### Triage Agents & Handoff

Agents-as-tools have a limited flow: a main agent calls another, which returns a result, and control flow resumes to the main agent. If we want the called agent to continue as the main, we can instead implement a *triage* pattern, where agents can hand off tasks to one another, and then have the called agent continue with the main conversation without returning control.

In [24]:
german_agent = Agent(
    name="german_agent",
    instructions="You only speak German",
    model=model
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
    model=model
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English",
    model=model
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[german_agent, spanish_agent, english_agent],
    model=model
)

In [25]:
msg = "Hi, i would like know more about your return policy."
triage_result = await Runner.run(triage_agent, msg)
mdprint(triage_result.final_output)

Sure! While the exact details can differ depending on the retailer or brand you’re dealing with, most return policies include the following common elements:

| **Item** | **Typical Requirement** |
|----------|--------------------------|
| **Time Frame** | Most stores allow returns within 30 days of purchase (some extend to 45 or 60 days for certain items). |
| **Condition** | Items usually need to be in **new, unused, and resalable condition** with all original tags, accessories, and packaging. |
| **Proof of Purchase** | A receipt, order confirmation email, or packing slip is generally required. Some places accept a credit‑card statement if the original receipt is unavailable. |
| **Proof of Identity** | A photo ID may be asked for, especially for high‑value items. |
| **Refund Method** | Refunds are typically issued to the original payment method (credit/debit card, PayPal, store credit, etc.). Some retailers offer a choice between cash, store credit, or an exchange. |
| **Exclusions & Exceptions** | Certain products—like intimate apparel, personal care items, perishable goods, custom‑made items, or software—may be non‑returnable or have a shorter return window. |
| **Return Process** | • **In‑store:** Bring the item, receipt, and ID to the customer‑service desk.<br>• **Online:** Log into your account, initiate a return request, print a prepaid shipping label (if applicable), and drop the package off at the designated carrier. |
| **Restocking Fees** | Some merchants charge a 10–20 % restocking fee for large or specialty items (e.g., electronics, furniture). |
| **Damaged/Defective Items** | If an item arrives damaged or is defective, most companies will cover return shipping and either replace the product or issue a full refund, regardless of the standard return window. |
| **International Orders** | Returns may be subject to additional customs paperwork, taxes, or duties, and the retailer may require the buyer to cover shipping costs. |

### What to Do Next
1. **Check the specific retailer’s website** – Look for a “Returns & Exchanges” or “Refund Policy” page.  
2. **Locate your receipt or order confirmation** – Having this handy speeds up the process.  
3. **Gather the item and all its accessories** – Make sure it’s clean and in its original packaging.  
4. **Contact customer service** if you’re unsure about any of the above points; they can confirm any store‑specific nuances (e.g., holiday return extensions, loyalty‑program perks, or special handling for large items).

If you let me know the name of the store or brand you’re dealing with, I can provide more tailored information!

In [26]:
msg = "Guten Tag, ich würde gern wissen wie ich eine Rücksendung erstelle."
triage_result = await Runner.run(triage_agent, msg)
mdprint(triage_result.final_output)

Guten Tag!

Gerne erkläre ich Ihnen, wie Sie eine Rücksendung erstellen können. Die genauen Schritte können je nach Onlineshop leicht variieren, aber im Allgemeinen läuft der Vorgang folgendermaßen ab:

---

## 1. Rückgabefrist prüfen
- **Frist:** In der Regel haben Sie 14 – 30 Tage ab Erhalt der Ware Zeit, um eine Rücksendung zu veranlassen.  
- **Bedingungen:** Die Artikel sollten unbenutzt, unbeschädigt und in der Originalverpackung sein (falls angegeben).

---

## 2. Kundenkonto öffnen
1. **Einloggen:** Gehen Sie auf die Website des Händlers und melden Sie sich mit Ihrer E‑Mail-Adresse und Ihrem Passwort an.  
2. **Bestellübersicht:** Navigieren Sie zu „Mein Konto“ → „Bestellungen“ (oder ähnliches).

---

## 3. Rücksendung starten
1. **Bestellung auswählen:** Finden Sie die Bestellung, die den zurückzusendenden Artikel enthält.  
2. **Rücksendung initiieren:** Klicken Sie auf den Button „Rücksendung/Umtausch starten“, „Artikel zurücksenden“ oder eine vergleichbare Option.  
3. **Grund angeben:** Wählen Sie den Grund für die Rücksendung aus dem Dropdown‑Menü (z. B. „Passform stimmt nicht“, „Artikel beschädigt“, „andere“).  
4. **Menge angeben:** Geben Sie an, wie viele Stück Sie zurücksenden möchten.

---

## 4. Rücksendeschein erstellen
- **Rücksendeetikett:** Der Shop stellt Ihnen in der Regel ein PDF‑Rücksendeetikett zur Verfügung. Dieses können Sie herunterladen und ausdrucken.  
- **Rücksendeformular:** Oft gibt es ein kurzes Formular, das Sie ausfüllen und dem Paket beilegen (z. B. Bestellnummer, Rücksende‑ID).

---

## 5. Paket vorbereiten
1. **Verpacken:** Legen Sie den Artikel, das ausgefüllte Rücksendeformular und ggf. das Originalzubehör in die Originalverpackung (oder eine vergleichbare Schutzverpackung).  
2. **Etikett anbringen:** Kleben Sie das ausgedruckte Rücksendeetikett gut sichtbar auf das Paket.  
3. **Adressfeld überprüfen:** Stellen Sie sicher, dass die Empfängeradresse vollständig und korrekt ist.

---

## 6. Paket abgeben / abholen lassen
- **Versanddienstleister:** Je nach Anbieter können Sie das Paket bei einer Filiale (z. B. DHL, Hermes, UPS, DPD) abgeben oder einen Abholservice nutzen.  
- **Quittung aufbewahren:** Nehmen Sie den Einlieferungsbeleg oder die Sendungsbestätigung (mit Sendungsnummer) für Ihren Nachweis entgegen.

---

## 7. Rückerstattung / Umtausch verfolgen
- **Benachrichtigung:** Sobald der Händler das zurückgesandte Paket erhalten und geprüft hat, erhalten Sie gewöhnlich eine E‑Mail über die weitere Vorgehensweise.  
- **Rückerstattung:** Bei einer Rückerstattung wird der Betrag meist dem ursprünglichen Zahlungsweg gutgeschrieben (Kreditkarte, PayPal, etc.).  
- **Umtausch:** Falls Sie einen Umtausch gewählt haben, wird die neue Ware nach Prüfung versendet.

---

## 8. Häufige Fragen (FAQ)

| Frage | Antwort |
|-------|----------|
| **Wie lange dauert die Rückerstattung?** | In der Regel 5‑10 Werktage nach Wareneingang, kann je nach Zahlungsanbieter variieren. |
| **Kann ich einen Artikel ohne Grund zurücksenden?** | Viele Händler erlauben das innerhalb der Rückgabefrist, manchmal jedoch nur gegen Rücksendegebühr. |
| **Was kostet das Rücksendeetikett?** | Häufig kostenfrei, bei manchen Angeboten wird die Rücksendekostenpauschale dem Kunden in Rechnung gestellt. |
| **Wie finde ich meine Rücksende‑ID?** | Sie steht auf dem Rücksendeetikett bzw. im Bestell‑ bzw. Rücksende‑E‑Mail. |
| **Was, wenn das Paket verloren geht?** | Nutzen Sie die Sendungsnummer, um den Versandstatus zu prüfen. Bei Verlust übernimmt meist der Versanddienstleister die Versicherung, sofern diese abgeschlossen wurde. |

---

### Tipp
**Screenshots machen:** Während Sie die Rücksendung online ausfüllen, kann es hilfreich sein, Screenshots von Bestell‑ und Rücksende‑Bestätigung zu speichern. Das ist besonders nützlich, falls es zu Unstimmigkeiten kommt.

---

Falls Sie noch Fragen zu einem bestimmten Händler oder zu einem speziellen Artikel haben, teilen Sie mir bitte den Namen des Shops oder die Bestellnummer mit – dann kann ich Ihnen gezieltere Anweisungen geben.

Viel Erfolg beim Rücksenden!  
Falls Sie weitere Unterstützung benötigen, stehe ich Ihnen gerne zur Verfügung.

In [27]:
triage_result

RunResult(input='Guten Tag, ich würde gern wissen wie ich eine Rücksendung erstelle.', new_items=[ReasoningItem(agent=Agent(name='triage_agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='Handoff to the appropriate agent based on the language of the request.', prompt=None, handoffs=[Agent(name='german_agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You only speak German', prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x105c19690>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], outp

### Guardrails

Guardrails are checks that run in parallel to the agent's execution. We discern between input guardrails and output guardrails.

Input guardrails are used to, for example:
- Check if input messages are off-topic
- Check that input messages don't violate any policies
- Take over control of the agent's execution if an unexpected input is detected

Output guardrails are used to, for example:
- Check if the output contains sensitive data
- Check if the output is a valid response to the user's message

In [28]:
@dataclass
class GuardrailCriterion:
    reasoning: str
    trigger: bool

In [29]:
from agents import input_guardrail, GuardrailFunctionOutput

input_guardrail_agent = Agent(
    name="Guardrail check",
    instructions=(
        "Check if the user is asking you to do their math homework."
        "Reply in the given structured format, conforming exactly to its specification: "
        f"{TypeAdapter(GuardrailCriterion).json_schema()}"
    ),
    model=model
)

@input_guardrail
async def math_guardrail(context, agent, input):
    """This is an input guardrail function, which happens to call an agent to check if the input
    is a math homework question.
    """
    result = await Runner.run(input_guardrail_agent, input, context=context.context)
    criterion = GuardrailCriterion(**json.loads(result.final_output))
    return GuardrailFunctionOutput(
        output_info=criterion.reasoning,
        tripwire_triggered=criterion.trigger
    )

In [30]:
from agents import output_guardrail

@output_guardrail
async def sensitive_data_check(context, agent, output):
    phone_number_in_response = "+49" in output

    return GuardrailFunctionOutput(
        output_info="Phone number in response!",
        tripwire_triggered=phone_number_in_response,
    )

In [31]:
from agents import InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered

agent = Agent(
    name="Friendly agent",
    instructions="You are a friendly helpful agent, eager to help the user with whatever they request.",
    input_guardrails=[math_guardrail],
    output_guardrails=[sensitive_data_check],
    model=model
)

async def call(prompt):
    try:
        result = await Runner.run(agent, prompt)
        print(result.final_output)
    except InputGuardrailTripwireTriggered:
        print("Sorry, I can't help you with your math homework.")
    except OutputGuardrailTripwireTriggered:
        print("Sorry, I can't provide you with sensitive data.")

In [32]:
await call("Please solve this equation for x: 4x^2 + 2x = 19")

Sorry, I can't help you with your math homework.


In [33]:
await call("Can you give me the phone number of the university of kassel?")

Sorry, I can't provide you with sensitive data.
